In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import itertools
import os, re
import nltk
from bs4 import BeautifulSoup
sns.set()
sns.set_context('talk')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [8.0, 6.0]
colours = itertools.cycle(sns.color_palette('dark'))
palettes = itertools.cycle([sns.color_palette('PuBuGn_d'), sns.color_palette('Oranges_d'), sns.color_palette('GnBu_d'), sns.color_palette('Reds_d'), sns.color_palette('Blues_d'), sns.color_palette('PuRd_d')])
import csv
import spacy
spacy.load('en_core_web_sm')
nlp = spacy.load('en')
from collections import defaultdict

In [3]:
path = './review_corpus/'
files = [ path+d+'/'+f for d in os.listdir(path) if os.path.isdir(path+d) for f in os.listdir(path+d) if f.endswith('.txt') ]
core_texts = []

def deHTML(comment):
    bs = BeautifulSoup(comment, 'lxml').text
    newlinetabs_removed = re.sub('[\n\t]', '', bs)
    extraws_removed = re.sub('\s\s+', ' ', newlinetabs_removed)
    return extraws_removed.strip()

for f in files:
    genre = f.split('review_corpus/')[1].split('/')[0]
    
    with open(f, 'r', encoding='latin1') as fo:
        fo.readline()
        text = fo.read()
        core_texts.append([genre, deHTML(text)])

In [4]:
flat_list = [i[1] for i in pd.DataFrame(core_texts).values]
genre_list = [i[0] for i in pd.DataFrame(core_texts).values]
        
tokens = []
lemma = []
pos = []
freq = 0
adj_count = 0

for index,doc in enumerate(nlp.pipe(flat_list, batch_size=50, n_threads=80)):
    print('still going:', index)
    if doc.is_parsed:
        tokens.append([n.text for n in doc])
        lemma.append([n.lemma_ for n in doc])
        pos.append([n.pos_ for n in doc])
    else:
        # to make sure the indices will line up
        tokens.append(None)
        lemma.append(None)
        pos.append(None)

still going: 0
still going: 1
still going: 2
still going: 3
still going: 4
still going: 5
still going: 6
still going: 7
still going: 8
still going: 9
still going: 10
still going: 11
still going: 12
still going: 13
still going: 14
still going: 15
still going: 16
still going: 17
still going: 18
still going: 19
still going: 20
still going: 21
still going: 22
still going: 23
still going: 24
still going: 25
still going: 26
still going: 27
still going: 28
still going: 29
still going: 30
still going: 31
still going: 32
still going: 33
still going: 34
still going: 35
still going: 36
still going: 37
still going: 38
still going: 39
still going: 40
still going: 41
still going: 42
still going: 43
still going: 44
still going: 45
still going: 46
still going: 47
still going: 48
still going: 49
still going: 50
still going: 51
still going: 52
still going: 53
still going: 54
still going: 55
still going: 56
still going: 57
still going: 58
still going: 59
still going: 60
still going: 61
still going: 62
st

In [5]:
adv_adj_dict = defaultdict(int)
adv_adj_genre_dict = defaultdict(int)
adj_genre_dict = defaultdict(int)
wordcount_dict = defaultdict(int)
genre_lists = []
adv_adj_list = []

for comment_index, comment in enumerate(pos):
    length = len(comment)
    genre = genre_list[comment_index] # TODO: handle None
    if genre is not None:
        wordcount_dict[genre] += length
        for word_index, part in enumerate(comment):
            genre_lists.append([genre, tokens[comment_index][word_index]])
            
            if (part == 'ADJ'):
                adj_count += 1
                adj_genre_dict[genre] += 1

            # for each adverb ending in -ly
            if (part == 'ADV') and tokens[comment_index][word_index][-2:] == 'ly':
                # if the next word is adj
                if (word_index+1 < length):
                    if (comment[word_index+1] == 'ADJ'):
                        # count it
                        freq = freq + 1
                        # dict key is 'word1 word2' in lowercase
                        key = tokens[comment_index][word_index].lower() + " " + tokens[comment_index][word_index+1].lower()
                        # add to the dict
                        adv_adj_dict[key] += 1
                        # count for the timestamp as well
                        adv_adj_genre_dict[genre] += 1
                        adv_adj_list.append([genre, key])

In [7]:
pos_list = []
for comment_index,text in enumerate(pos):
    genre = genre_list[comment_index] # TODO: handle None
    if genre is not None:
        for word_index,part in enumerate(text):
            pos_list.append([genre, comment_index, tokens[comment_index][word_index], part])

In [12]:
pd.DataFrame(pos_list).rename(columns={0 : 'text_num', 1 : 'token', 2 : 'pos'}).to_csv('Movies_pos.csv')

[['COMPUTERS', 399, 'it', 'PRON'],
 ['COMPUTERS', 399, "'s", 'VERB'],
 ['COMPUTERS', 399, 'really', 'ADV'],
 ['COMPUTERS', 399, 'fast', 'ADJ'],
 ['COMPUTERS', 399, '.', 'PUNCT']]

In [28]:
adv_adj_list

[['BOOKS', 'completely sympathetic'],
 ['BOOKS', 'fairly surprised'],
 ['BOOKS', 'enormously repetitive'],
 ['BOOKS', 'completely unsympathetic'],
 ['BOOKS', 'apparently uninterested'],
 ['BOOKS', 'highly favorable'],
 ['BOOKS', 'inherently monstrous'],
 ['BOOKS', 'obviously fake'],
 ['BOOKS', 'increasingly frustrated'],
 ['BOOKS', 'mostly unexpected'],
 ['BOOKS', 'considerably wittier'],
 ['BOOKS', 'sexually straightforward'],
 ['BOOKS', 'truly exceptional'],
 ['BOOKS', 'fully supportive'],
 ['BOOKS', 'completely reverent'],
 ['BOOKS', 'thoroughly enjoyable'],
 ['BOOKS', 'completely different'],
 ['BOOKS', 'only sporadic'],
 ['BOOKS', 'only familiar'],
 ['BOOKS', 'definitely his'],
 ['BOOKS', 'overly choppy'],
 ['BOOKS', 'deeply spiritual'],
 ['BOOKS', 'highly religious'],
 ['BOOKS', 'hardly shy'],
 ['BOOKS', 'delightfully clever'],
 ['BOOKS', 'extremely insightful'],
 ['BOOKS', 'easily able'],
 ['BOOKS', 'fairly obvious'],
 ['BOOKS', 'entirely appropriate'],
 ['BOOKS', 'wonderfully d

In [29]:
from collections import Counter
adv_adj_list_counter = Counter([pair[0]+','+pair[1] for pair in adv_adj_list])
adv_adj_list = [[k.split(',')[0], k.split(',')[1], v] for (k,v) in adv_adj_list_counter.items()]

In [30]:
wordcount_dict

defaultdict(int,
            {'BOOKS': 29083,
             'CARS': 54910,
             'COMPUTERS': 48137,
             'COOKWARE': 24684,
             'HOTELS': 36985,
             'MOVIES': 36838,
             'MUSIC': 50010,
             'PHONES': 16238})

In [31]:
adj_genre_dict

defaultdict(int,
            {'BOOKS': 2513,
             'CARS': 5341,
             'COMPUTERS': 4015,
             'COOKWARE': 2234,
             'HOTELS': 3306,
             'MOVIES': 3349,
             'MUSIC': 4588,
             'PHONES': 1426})

In [32]:
adv_adj_genre_dict

defaultdict(int,
            {'BOOKS': 57,
             'CARS': 155,
             'COMPUTERS': 90,
             'COOKWARE': 40,
             'HOTELS': 96,
             'MOVIES': 99,
             'MUSIC': 127,
             'PHONES': 35})

In [33]:
adv_adj_dict

defaultdict(int,
            {'absolutely amazing': 2,
             'absolutely awful': 1,
             'absolutely beautiful': 1,
             'absolutely breathtaking': 1,
             'absolutely charming': 1,
             'absolutely dreamy': 1,
             'absolutely featureless': 1,
             'absolutely frightening': 1,
             'absolutely gorgeous': 2,
             'absolutely horrifying': 1,
             'absolutely incredible': 1,
             'absolutely obnoxious': 1,
             'absolutely perfect': 1,
             'absolutely regal': 1,
             'absolutely stellar': 1,
             'absolutely stunning': 1,
             'absolutely sure': 1,
             'absolutely terrible': 2,
             'absolutely unacceptable': 1,
             'absolutely useless': 1,
             'absurdly thin': 1,
             'abundantly clear': 1,
             'actually able': 1,
             'actually beautiful': 1,
             'actually concerned': 1,
             'actuall

In [34]:
genre_lists

[['BOOKS', 'The'],
 ['BOOKS', 'story'],
 ['BOOKS', 'seems'],
 ['BOOKS', 'simple'],
 ['BOOKS', 'enough'],
 ['BOOKS', 'from'],
 ['BOOKS', 'the'],
 ['BOOKS', 'beginning'],
 ['BOOKS', '-'],
 ['BOOKS', 'a'],
 ['BOOKS', 'priest'],
 ['BOOKS', 'in'],
 ['BOOKS', 'a'],
 ['BOOKS', 'small'],
 ['BOOKS', 'Italian'],
 ['BOOKS', 'town'],
 ['BOOKS', 'takes'],
 ['BOOKS', 'a'],
 ['BOOKS', 'ghastly'],
 ['BOOKS', 'confession'],
 ['BOOKS', 'from'],
 ['BOOKS', 'a'],
 ['BOOKS', 'local'],
 ['BOOKS', 'doctor'],
 ['BOOKS', '.'],
 ['BOOKS', 'Upon'],
 ['BOOKS', 'hearing'],
 ['BOOKS', 'the'],
 ['BOOKS', 'confession'],
 ['BOOKS', 'he'],
 ['BOOKS', 'drops'],
 ['BOOKS', 'everything'],
 ['BOOKS', ','],
 ['BOOKS', 'closes'],
 ['BOOKS', 'up'],
 ['BOOKS', 'his'],
 ['BOOKS', 'church'],
 ['BOOKS', 'and'],
 ['BOOKS', 'high'],
 ['BOOKS', '-'],
 ['BOOKS', 'tails'],
 ['BOOKS', 'it'],
 ['BOOKS', 'for'],
 ['BOOKS', 'the'],
 ['BOOKS', 'Vatican'],
 ['BOOKS', 'to'],
 ['BOOKS', 'relate'],
 ['BOOKS', 'this'],
 ['BOOKS', 'unholy'],
 ['

In [49]:
print('Number of tokens')
print(sum([len(sublist) for sublist in pos]))

types = set([i for sublist in tokens for i in sublist])
print('Overall lexical diversity')
print(len(types) / len(tokens))

Number of tokens
296885
Overall lexical diversity
52.83


In [35]:
# dictionary - each key is a tuple with the advadj pair and the value is the count
print("Freq: " +str(freq))
print("Adjective count: " +str(adj_count))

with open('adj_genre_dict.csv', 'w+') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in adj_genre_dict.items():
       writer.writerow([key, value])

with open('advadj_genre_dict.csv', 'w+') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in adv_adj_genre_dict.items():
       writer.writerow([key, value])

with open('advadj_dict.csv', 'w+') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in adv_adj_dict.items():
       writer.writerow([key, value])
    
with open('genre_wordcounts.csv', 'w+') as f:
    writer = csv.writer(f)
    for key, value in wordcount_dict.items():
       writer.writerow([key, value])

Freq: 1398
Adjective count: 53544


In [36]:
import re
df = pd.DataFrame(adv_adj_list).rename(columns={0 : 'genre', 1 : 'pair', 2 : 'count'})
wordcounts = pd.DataFrame(list(wordcount_dict.items())).rename(columns={0 : 'genre', 1 : 'wordcount'})
merged = df.merge(wordcounts, on='genre')
merged['freqmil'] = merged['count'] / merged['wordcount'] * 1000000

In [37]:
grouped = merged.groupby('genre')
for i in grouped.groups:
    print(i)
    filename = re.sub('\s+', '_', re.sub('/', '', i))
    grouped.get_group(i)[['pair', 'count', 'freqmil']].to_csv(filename + '.txt', index=False, float_format='%.4f')

HOTELS
COMPUTERS
MOVIES
BOOKS
PHONES
COOKWARE
MUSIC
CARS


In [39]:
genre_tokens = pd.DataFrame(genre_lists).rename(columns={0: 'genre', 1 : 'tokens'})

In [48]:
pd.DataFrame(genre_tokens.drop_duplicates().groupby('genre').count()['tokens'] / genre_tokens.groupby('genre').count()['tokens']).to_csv('lexical_diversity.csv')

In [50]:
with open("overview.txt", "w+") as f:
    f.write("Frequency: " + str(freq) + "\n")
    f.write("Adjective count: " + str(adj_count) + "\n")
    f.write("Number of tokens: " + str(sum([len(sublist) for sublist in pos])) + "\n")
    types = set([i for sublist in tokens for i in sublist])
    f.write("Overall lexical diversity: " + str(len(types) / len(tokens)) + "\n")

In [66]:
sum([len(sublist) for sublist in pos])

59407779